In [4]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib
from sklearn.model_selection import train_test_split

# conectar base de datos
conn=sql.connect('data\\movies2.db')
cur=conn.cursor()

# ver tablas disponibles en base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('f_ratings',),
 ('reco',)]

# <b>1. Sistemas basados en popularidad </b>
El sistema recomendará estas películas a todos los usuarios, destacando lo más popular en el momento. Se extraen las 10 películas más vistas y mejor calificadas (≥ 4). Se actualiza diariamente.<p>
El sistema recomendará las películas más populares a todos los usuarios, destacando lo que es tendencia en la plataforma. Las recomendaciones se basan en la selección de las 10 películas mejor calificadas del catálogo, con una calificación promedio de al menos 4. Este sistema está pensado para reflejar lo que la mayoría de los usuarios están viendo y disfrutando. <br>
La frecuencia de actualización es diaria, lo que asegura que el catálogo de recomendaciones esté siempre al día con los cambios en la popularidad de las películas. Esto resulta útil cuando se producen estrenos importantes o cuando una película comienza a generar gran interés de forma inesperada.

## 1.1. Películas con mejor calificación

In [9]:
# películas vistas por más de 20 personas y con calificación mayor a 4
pd.read_sql("""
            SELECT movie_title,
                   avg(rating) as avg_rat,
                   count(*) as movie_num
            FROM f_ratings
            WHERE rating > 4  -- Restringir a calificaciones mayores a 4
            GROUP BY movie_title
            HAVING count(*) >= 20
            ORDER BY avg_rat DESC, movie_num DESC
            LIMIT 10
            """, conn)

,movie_title,avg_rat,movie_num
0,Babe (1995),4.962963,27
1,Jaws (1975),4.953125,32
2,Speed (1994),4.934783,23
3,"Bridge on the River Kwai, The (1957)",4.928571,21
4,Brazil (1985),4.925000,20
5,Apocalypse Now (1979),4.922222,45
6,2001: A Space Odyssey (1968),4.921875,32
7,"Fugitive, The (1993)",4.920000,50
8,Airplane! (1980),4.920000,25
9,Star Wars: Episode IV - A New Hope (1977),4.919048,105


## 1.2. Película mejor calificada por año

In [10]:
# los mejores calificados por año publicación
pd.read_sql("""SELECT Año, movie_title, num_vistas
                FROM (
                    SELECT
                        strftime('%Y', fecha_nueva) AS Año,
                        movie_title,
                        ROW_NUMBER() OVER(PARTITION BY strftime('%Y', fecha_nueva) ORDER BY COUNT(*) DESC) AS row_num,
                        MAX(COUNT(*)) OVER(PARTITION BY strftime('%Y', fecha_nueva)) AS num_vistas
                    FROM f_ratings
                    GROUP BY strftime('%Y', fecha_nueva), movie_title) AS subquery
                WHERE row_num = 1
                ORDER BY Año;""", conn)

,Año,movie_title,num_vistas
0,1996,Batman (1989),69
1,1997,Independence Day (a.k.a. ID4) (1996),21
2,1998,"Silence of the Lambs, The (1991)",5
3,1999,Saving Private Ryan (1998),12
4,2000,American Beauty (1999),28
5,2001,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",12
6,2002,"Silence of the Lambs, The (1991)",10
7,2003,"Lord of the Rings: The Two Towers, The (2002)",14
8,2004,Austin Powers: The Spy Who Shagged Me (1999),9
9,2005,"Sixth Sense, The (1999)",14
